In [14]:
import argparse
import logging
import os
import pprint
import mxnet as mx
import numpy as np

from rcnn.dataset import *
from rcnn.core import callback, metric
from rcnn.core.loader import AnchorLoader
from rcnn.core.module import MutableModule
from rcnn.utils.load_model import load_param
from rcnn.symbol.symbol_vgg import *
from rcnn.config import config

In [15]:
# set up logger
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

print 'set config'
# setup config
config.TRAIN.BATCH_IMAGES = 1
config.TRAIN.BATCH_ROIS = 128
config.TRAIN.END2END = True
config.TRAIN.BBOX_3D = True
config.TRAIN.BBOX_NORMALIZATION_PRECOMPUTED = True
config.TRAIN.BG_THRESH_LO = 0.0

print 'load symbol'
# load symbol
sym = eval('get_vgg_train')()
feat_sym = sym.get_internals()['rpn_cls_score_output']

ctx=[mx.gpu(4)]
batch_size = len(ctx)
input_batch_size = config.TRAIN.BATCH_IMAGES * batch_size
pprint.pprint(config)
imdb = eval('Kitti')('train_ry_alpha_car_only', 'data', 'data/kitti')
roidb=imdb.gt_roidb()

set config
load symbol
{'ANCHOR_RATIOS': [0.5, 1, 2],
 'ANCHOR_SCALES': [8, 16, 32, 48],
 'EPS': 1e-14,
 'FIXED_PARAMS': ['conv1', 'conv2', 'gamma', 'beta'],
 'FIXED_PARAMS_FINETUNE': ['conv1', 'conv2', 'gamma', 'beta'],
 'IMAGE_STRIDE': 0,
 'INVALID_ORI': -10000,
 'NUM_ANCHORS': 12,
 'NUM_CLASSES': 4,
 'PI': 3.141592653,
 'PIXEL_MEANS': array([[[ 123.68 ,  116.779,  103.939]]]),
 'RCNN_FEAT_SRTIDE': 16,
 'RPN_FEAT_STRIDE': 16,
 'RY_CLASSES': 72,
 'SCALES': [(752, 2500)],
 'TEST': {'BATCH_IMAGES': 1,
          'CXX_PROPOSAL': False,
          'HAS_RPN': False,
          'NMS': 0.3,
          'RPN_MIN_SIZE': 16,
          'RPN_NMS_THRESH': 0.7,
          'RPN_POST_NMS_TOP_N': 300,
          'RPN_PRE_NMS_TOP_N': 6000},
 'TRAIN': {'BATCH_IMAGES': 1,
           'BATCH_ROIS': 128,
           'BATCH_SIZE': 1,
           'BBOX_3D': True,
           'BBOX_INSIDE_WEIGHTS': array([ 1.,  1.,  1.,  1.]),
           'BBOX_MEANS': [0.0, 0.0, 0.0, 0.0],
           'BBOX_NORMALIZATION_PRECOMPUTED': Tr

In [16]:
train_data = AnchorLoader(feat_sym, roidb, batch_size=input_batch_size, shuffle=True,
                              ctx=ctx, work_load_list=None)

752


KeyError: 'gt_dims'

In [34]:
a = roidb[10]['boxes']

In [33]:
roidb[10]['gt_classes']

array([1, 1, 1], dtype=int32)

In [39]:
roidb[0]

{'boxes': array([[ 387.63000488,  181.53999329,  423.80999756,  203.11999512]], dtype=float32),
 'flipped': False,
 'gt_classes': array([1], dtype=int32),
 'gt_overlaps': array([[ 0.,  1.,  0.,  0.]], dtype=float32),
 'height': 375,
 'image': 'data/kitti/images/000001.png',
 'max_classes': array([1]),
 'max_overlaps': array([ 1.], dtype=float32),
 'orientation_alpha': array([ 1.85000002], dtype=float32),
 'orientation_ry': array([ 1.57000005], dtype=float32),
 'width': 1242}